# Imports

In [61]:
from datetime import timedelta
import pandas as pd
import yaml

# Config

In [4]:
%cd ..

/Users/adriantrujillo/truji/mlops-bootcamp-e7


/Users/adriantrujillo/truji/mlops-bootcamp-e7/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
with open("params.yaml") as conf_file:
    config = yaml.safe_load(conf_file)

# Load and clean the data

In [66]:
df = pd.read_csv(config["data_load"]["dataset_csv"], encoding="ISO-8859-1")

In [67]:
from src.data_preprocess.clean_data import clean_data

df = clean_data(df)

In [68]:
# Replace all column names for snake case
df.rename(columns=lambda x: x.lower(), inplace=True)
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [69]:
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [70]:
df["total_price"] = df["quantity"] * df["unitprice"]
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34


In [71]:
df_uk = df[df["country"] == "United Kingdom"]

# Recency

In [73]:
min_invoice_date = min(df_uk["invoicedate"])
min_invoice_date

'1/10/2011 10:32'

In [74]:
max_invoice_date = max(df_uk["invoicedate"])
max_invoice_date

'9/9/2011 9:52'

In [75]:
df_uk["last_purchase_date"] = df_uk.groupby("customerid")["invoicedate"].transform(max)

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2499455468.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_uk['last_purchase_date'] = df_uk.groupby('customerid')['invoicedate'].transform(max)
/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2499455468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['last_purchase_date'] = df_uk.groupby('customerid')['invoicedate'].transform(max)


In [76]:
df_uk["last_purchase_date"] = pd.to_datetime(df_uk["last_purchase_date"]).dt.date

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2544061447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['last_purchase_date'] = pd.to_datetime(df_uk['last_purchase_date']).dt.date


In [77]:
df_uk["last_purchase_date"] = pd.to_datetime(df_uk["last_purchase_date"])
df_uk["invoicedate"] = pd.to_datetime(df_uk["invoicedate"])

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/1704937806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['last_purchase_date'] = pd.to_datetime(df_uk['last_purchase_date'])
/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/1704937806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['invoicedate'] = pd.to_datetime(df_uk['invoicedate'])


In [78]:
df_uk["ref_date"] = df_uk["invoicedate"].max() + timedelta(days=7)

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2060611127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['ref_date'] = df_uk['invoicedate'].max() + timedelta(days=7)


In [80]:
df_uk["ref_date"] = df_uk["ref_date"].dt.date

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/650992971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['ref_date'] = df_uk['ref_date'].dt.date


In [83]:
df_uk["ref_date"] = pd.to_datetime(df_uk["ref_date"])

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/3719987255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['ref_date'] = pd.to_datetime(df_uk['ref_date'])


In [84]:
df_uk["date"] = pd.to_datetime(df_uk["invoicedate"])
df_uk["date"] = df_uk["date"].dt.date

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/1014098543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['date'] = pd.to_datetime(df_uk['invoicedate'])
/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/1014098543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['date'] = df_uk['date'].dt.date


In [85]:
customer_recency = pd.DataFrame(df_uk.groupby("customerid", as_index=False).date.max())
customer_recency.head()

,customerid,date
0,12346,2011-01-18
1,12747,2011-12-07
2,12748,2011-12-09
3,12749,2011-12-06
4,12820,2011-12-06


In [86]:
df_uk["customer_recency"] = df_uk["ref_date"] - df_uk["last_purchase_date"]
df_uk[["customerid", "last_purchase_date", "ref_date", "customer_recency"]]

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2118949637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["customer_recency"] = df_uk["ref_date"] - df_uk["last_purchase_date"]


,customerid,last_purchase_date,ref_date,customer_recency
0,17850,2010-12-02,2011-12-16,379 days
1,17850,2010-12-02,2011-12-16,379 days
2,17850,2010-12-02,2011-12-16,379 days
3,17850,2010-12-02,2011-12-16,379 days
4,17850,2010-12-02,2011-12-16,379 days
...,...,...,...,...
541889,15804,2011-09-06,2011-12-16,101 days
541890,13113,2011-08-09,2011-12-16,129 days
541891,13113,2011-08-09,2011-12-16,129 days
541892,13113,2011-08-09,2011-12-16,129 days


In [87]:
df_uk["recency2"] = pd.to_numeric(df_uk["customer_recency"].dt.days.astype("int64"))
df_uk[["customerid", "last_purchase_date", "ref_date", "customer_recency", "recency2"]]

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/2544459196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk['recency2'] = pd.to_numeric(df_uk['customer_recency'].dt.days.astype('int64'))


,customerid,last_purchase_date,ref_date,customer_recency,recency2
0,17850,2010-12-02,2011-12-16,379 days,379
1,17850,2010-12-02,2011-12-16,379 days,379
2,17850,2010-12-02,2011-12-16,379 days,379
3,17850,2010-12-02,2011-12-16,379 days,379
4,17850,2010-12-02,2011-12-16,379 days,379
...,...,...,...,...,...
541889,15804,2011-09-06,2011-12-16,101 days,101
541890,13113,2011-08-09,2011-12-16,129 days,129
541891,13113,2011-08-09,2011-12-16,129 days,129
541892,13113,2011-08-09,2011-12-16,129 days,129


In [88]:
customer_recency = df_uk.groupby("customerid", as_index=False)["recency2"].mean()
customer_recency.rename(columns={"recency2": "Recency"}, inplace=True)
customer_recency.sort_values(by="Recency", ascending=False).head()

,customerid,Recency
1538,15012,380.0
3543,17760,380.0
2307,16048,380.0
1662,15165,380.0
3452,17643,380.0


In [90]:
df_uk.drop(["last_purchase_date"], axis=1, inplace=True)

/var/folders/1w/8crwsgs93wv2lcm5cbm6_tsm0000gn/T/ipykernel_63133/1685787660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk.drop(['last_purchase_date'], axis = 1, inplace=True)


# Frequency

In [91]:
dfUK_copy = df_uk.copy()

In [92]:
customer_frequency = dfUK_copy.groupby("customerid", as_index=False)[
    "invoiceno"
].nunique()
customer_frequency.rename(columns={"invoiceno": "Frequency"}, inplace=True)
customer_frequency.sort_values(by="Frequency", ascending=False)

,customerid,Frequency
2,12748,209
3593,17841,124
191,13089,97
1267,14606,93
1771,15311,91
...,...,...
953,14165,1
955,14171,1
957,14174,1
2576,16415,1


# Monetary

In [93]:
customer_monetary = dfUK_copy.groupby("customerid", as_index=False)["total_price"].sum()
customer_monetary.rename(columns={"total_price": "Monetary"}, inplace=True)
customer_monetary.sort_values(by="Monetary", ascending=False).head()

,customerid,Monetary
3783,18102,259657.30
3314,17450,194390.79
2598,16446,168472.50
3356,17511,91062.38
2294,16029,80850.84


# RFM Score

In [94]:
customer_rfm = pd.merge(
    pd.merge(customer_recency, customer_frequency, on="customerid"),
    customer_monetary,
    on="customerid",
)
customer_rfm.head()

,customerid,Recency,Frequency,Monetary
0,12346,332.0,1,77183.60
1,12747,116.0,11,4196.01
2,12748,98.0,209,33053.19
3,12749,137.0,5,4090.88
4,12820,81.0,4,942.34


In [56]:
customer_rfm.describe()

,Recency,Frequency,Monetary
count,3920.000000,3920.000000,3920.000000
mean,145.742602,4.246429,1858.424654
std,92.716569,7.199202,7478.631256
min,7.000000,1.000000,3.750000
25%,82.000000,1.000000,298.185000
50%,116.000000,2.000000,644.975000
75%,203.000000,5.000000,1571.285000
max,380.000000,209.000000,259657.300000


In [98]:
quantiles = customer_rfm[["Recency", "Frequency", "Monetary"]].quantile(
    q=[0.25, 0.50, 0.75]
)
quantiles

,Recency,Frequency,Monetary
0.25,82.0,1.0,298.185
0.50,116.0,2.0,644.975
0.75,203.0,5.0,1571.285


In [99]:
def recency_scoring(rfm):
    if rfm.Recency <= 24.0:
        recency_score = 4
    elif rfm.Recency <= 57.0:
        recency_score = 3
    elif rfm.Recency <= 149.0:
        recency_score = 2
    else:
        recency_score = 1
    return recency_score


customer_rfm["Recency_Score"] = customer_rfm.apply(recency_scoring, axis=1)
customer_rfm.sample(10)

,customerid,Recency,Frequency,Monetary,Recency_Score
1878,15458,32.0,1,161.64,3
2664,16536,117.0,1,548.12,2
1988,15607,343.0,1,104.76,1
2243,15958,81.0,1,104.35,2
262,13183,93.0,9,3957.78,2
613,13682,194.0,1,59.50,1
1316,14675,23.0,1,596.36,4
2526,16352,24.0,2,494.44,4
3884,18235,78.0,3,1796.48,2
2884,16834,234.0,1,413.46,1


In [100]:
def frequency_scoring(rfm):
    if rfm.Frequency >= 10.0:
        frequency_score = 4
    elif rfm.Frequency >= 5.0:
        frequency_score = 3
    elif rfm.Frequency >= 2.0:
        frequency_score = 2
    else:
        frequency_score = 1
    return frequency_score


customer_rfm["Frequency_Score"] = customer_rfm.apply(frequency_scoring, axis=1)
customer_rfm.sample(10)

,customerid,Recency,Frequency,Monetary,Recency_Score,Frequency_Score
2819,16753,65.0,1,223.92,2,1
2965,16948,145.0,3,435.02,2,2
940,14148,240.0,1,157.20,1,1
3772,18086,281.0,1,101.34,1,1
1396,14789,35.0,1,197.64,3,1
3481,17677,89.0,30,16345.38,2,4
440,13430,93.0,2,679.45,2,2
750,13882,9.0,5,1191.13,4,3
2666,16542,60.0,1,119.62,2,1
2501,16318,42.0,1,328.15,3,1


In [101]:
def monetary_scoring(rfm):
    if rfm.Monetary >= 1571.0:
        monetary_score = 4
    elif rfm.Monetary >= 645.0:
        monetary_score = 3
    elif rfm.Monetary >= 298.0:
        monetary_score = 2
    else:
        monetary_score = 1
    return monetary_score


customer_rfm["Monetary_Score"] = customer_rfm.apply(monetary_scoring, axis=1)
customer_rfm.sample(10)

,customerid,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
1423,14825,96.0,12,2221.01,2,4,4
920,14121,10.0,4,2780.15,4,2,4
1023,14264,161.0,2,333.97,1,2,2
3597,17849,225.0,4,605.80,1,2,2
1013,14247,285.0,1,136.77,1,1,1
2282,16015,10.0,3,704.55,4,2,3
1382,14766,87.0,6,2473.37,2,3,4
109,12970,14.0,4,405.47,4,2,2
113,12977,163.0,1,39.24,1,1,1
3091,17126,159.0,5,989.45,1,3,3


In [102]:
def rfm_scoring(customer):
    return (
        str(int(customer["Recency_Score"]))
        + str(int(customer["Frequency_Score"]))
        + str(int(customer["Monetary_Score"]))
    )


customer_rfm["Customer_RFM_Score"] = customer_rfm.apply(rfm_scoring, axis=1)
customer_rfm.sample(8)

,customerid,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,Customer_RFM_Score
1071,14340,225.0,1,134.70,1,1,1,111
1062,14327,170.0,1,311.65,1,1,2,112
1556,15034,77.0,8,4274.16,2,3,4,234
3750,18059,30.0,3,306.46,3,2,2,322
3641,17900,197.0,2,112.20,1,2,1,121
1933,15532,138.0,4,1564.73,2,2,3,223
2025,15657,29.0,1,30.00,3,1,1,311
407,13381,96.0,6,3639.31,2,3,4,234


In [103]:
customer_rfm["RFM_Label"] = (
    customer_rfm["Recency_Score"]
    + customer_rfm["Frequency_Score"]
    + customer_rfm["Monetary_Score"]
)

customer_rfm.sample(8)

,customerid,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,Customer_RFM_Score,RFM_Label
443,13435,82.0,7,3809.89,2,3,4,234,9
978,14206,246.0,1,328.16,1,1,2,112,4
469,13475,198.0,4,1431.78,1,2,3,123,6
1973,15587,96.0,4,913.68,2,2,3,223,7
172,13061,79.0,1,248.61,2,1,1,211,4
1798,15348,79.0,1,158.04,2,1,1,211,4
2512,16337,43.0,1,151.05,3,1,1,311,5
2894,16847,50.0,1,193.44,3,1,1,311,5


In [104]:
segments = {
    "Customer_Segment": [
        "Champion",
        "Top Loyal Customer",
        "Loyal Customer",
        "Top Recent Customer",
        "Recent Customer",
        "Top Customer Needed Attention",
        "Customer Needed Attention",
        "Top Lost Customer",
        "Lost Customer",
    ],
    "RFM": [
        "(2|3|4)-(4)-(4)",
        "(3)-(1|2|3|4)-(3|4)",
        "(3)-(1|2|3|4)-(1|2)",
        "(4)-(1|2|3|4)-(3|4)",
        "(4)-(1|2|3|4)-(1|2)",
        "(2|3)-(1|2|3|4)-(3|4)",
        "(2|3)-(1|2|3|4)-(1|2)",
        "(1)-(1|2|3|4)-(3|4)",
        "(1)-(1|2|3|4)-(1|2)",
    ],
}

pd.DataFrame(segments)

,Customer_Segment,RFM
0,Champion,(2|3|4)-(4)-(4)
1,Top Loyal Customer,(3)-(1|2|3|4)-(3|4)
2,Loyal Customer,(3)-(1|2|3|4)-(1|2)
3,Top Recent Customer,(4)-(1|2|3|4)-(3|4)
4,Recent Customer,(4)-(1|2|3|4)-(1|2)
5,Top Customer Needed Attention,(2|3)-(1|2|3|4)-(3|4)
6,Customer Needed Attention,(2|3)-(1|2|3|4)-(1|2)
7,Top Lost Customer,(1)-(1|2|3|4)-(3|4)
8,Lost Customer,(1)-(1|2|3|4)-(1|2)


In [105]:
def categorizer(rfm):
    if (rfm[0] in ["2", "3", "4"]) & (rfm[1] in ["4"]) & (rfm[2] in ["4"]):
        rfm = "Champion"

    elif (rfm[0] in ["3"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["3", "4"]):
        rfm = "Top Loyal Customer"

    elif (rfm[0] in ["3"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["1", "2"]):
        rfm = "Loyal Customer"

    elif (rfm[0] in ["4"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["3", "4"]):
        rfm = "Top Recent Customer"

    elif (rfm[0] in ["4"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["1", "2"]):
        rfm = "Recent Customer"

    elif (
        (rfm[0] in ["2", "3"])
        & (rfm[1] in ["1", "2", "3", "4"])
        & (rfm[2] in ["3", "4"])
    ):
        rfm = "Top Customer Needed Attention"

    elif (
        (rfm[0] in ["2", "3"])
        & (rfm[1] in ["1", "2", "3", "4"])
        & (rfm[2] in ["1", "2"])
    ):
        rfm = "Customer Needed Attention"

    elif (rfm[0] in ["1"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["3", "4"]):
        rfm = "Top Lost Customer"

    elif (rfm[0] in ["1"]) & (rfm[1] in ["1", "2", "3", "4"]) & (rfm[2] in ["1", "2"]):
        rfm = "Lost Customer"

    return rfm

In [106]:
customer_rfm["Customer_Category"] = customer_rfm["Customer_RFM_Score"].apply(
    categorizer
)
customer_rfm

,customerid,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,Customer_RFM_Score,RFM_Label,Customer_Category
0,12346,332.0,1,77183.60,1,1,4,114,6,Top Lost Customer
1,12747,116.0,11,4196.01,2,4,4,244,10,Champion
2,12748,98.0,209,33053.19,2,4,4,244,10,Champion
3,12749,137.0,5,4090.88,2,3,4,234,9,Top Customer Needed Attention
4,12820,81.0,4,942.34,2,2,3,223,7,Top Customer Needed Attention
...,...,...,...,...,...,...,...,...,...,...
3915,18280,284.0,1,180.60,1,1,1,111,3,Lost Customer
3916,18281,187.0,1,80.82,1,1,1,111,3,Lost Customer
3917,18282,133.0,2,178.05,2,2,1,221,5,Customer Needed Attention
3918,18283,102.0,16,2045.53,2,4,4,244,10,Champion
